# SANS Parity Tests - Comprehensive Comparison

This notebook runs parity tests to identify if SANS performance issues are in the weighting mechanism or deeper in the implementation.

## Test Configurations:
1. **Baseline iREM** - Original implementation without SANS
2. **SANS Normal** - SANS with α=2.0, K=16, temperature schedule
3. **SANS Parity (α=0)** - SANS with uniform weighting (α=0, K=16)

## Key Questions:
- Does SANS(α=0) match baseline performance? (Should be within ±1-2%)
- If not, the SANS codepath has structural differences beyond weighting
- Are the negative terms computed equivalently?

## 1. Setup Environment

In [ ]:
# Check GPU availability
import torch
import os
import sys
import io
from contextlib import redirect_stdout

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    print(f"GPU Name: {gpu_name}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    
    if 'T4' in gpu_name:
        print("✓ T4 GPU detected")
        batch_size = 256
    elif 'V100' in gpu_name:
        print("✓ V100 GPU detected")
        batch_size = 512
    else:
        print(f"✓ {gpu_name} detected")
        batch_size = 128
else:
    print("⚠ No GPU detected - running on CPU (will be slower)")
    batch_size = 32

print(f"\nUsing batch size: {batch_size}")

In [ ]:
# Clone repository or use existing files
!rm -rf energy-based-model
!git clone https://github.com/mdkrasnow/energy-based-model.git

import os
if not os.path.exists('energy-based-model'):
    print("Repository not found. Please either:")
    print("1. Update the git clone URL with your repository")
    print("2. Upload the energy-based-model folder to Colab")
    os.makedirs('energy-based-model', exist_ok=True)

os.chdir('energy-based-model')
print(f"Current directory: {os.getcwd()}")

In [ ]:
# Install dependencies
!pip install -q accelerate ema-pytorch einops tabulate tqdm matplotlib seaborn pandas
print("✓ Dependencies installed")

## 2. Import Modules and Configure

In [ ]:
# Set environment variables
os.environ['OPENBLAS_NUM_THREADS'] = '1'
os.environ['NUMEXPR_NUM_THREADS'] = '1'
os.environ['MKL_NUM_THREADS'] = '1'

# Import required modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json
import time
from typing import Dict, List, Optional
from tabulate import tabulate

# Import training modules
from diffusion_lib.denoising_diffusion_pytorch_1d import GaussianDiffusion1D, Trainer1D
from models import EBM, DiffusionWrapper
from dataset import Inverse, Addition, LowRankDataset
from irem_lib.irem import Trainer1D as iREMTrainer1D

print("✓ Modules imported successfully")

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

In [ ]:
# Base configuration
BASE_CONFIG = {
    # Dataset parameters
    'dataset': 'inverse',
    'rank': 10,
    'ood': False,
    
    # Model parameters
    'model': 'mlp',
    'diffusion_steps': 10,
    
    # Training parameters
    'batch_size': batch_size,
    'learning_rate': 1e-4,
    'num_steps': 5000,  # Run for 5k steps
    'val_every': 250,
    'save_every': 1000,
    
    # Common parameters
    'supervise_energy_landscape': True,
    'use_innerloop_opt': False,
    'data_workers': 2,
    'ema_decay': 0.995,
    'gradient_accumulate_every': 1,
    'amp': False,
    
    # Base results directory
    'results_dir': 'parity_test_results',
}

# Test configurations for parity testing
TEST_CONFIGS = {
    'baseline': {
        'sans_enabled': False,
        'description': 'Baseline iREM (no SANS)',
        'color': 'blue',
        'marker': 'o'
    },
    'sans_normal': {
        'sans_enabled': True,
        'sans_temp': 2.0,
        'sans_num_negs': 16,
        'sans_temp_schedule': True,
        'description': 'SANS (α=2.0, K=16, scheduled)',
        'color': 'red',
        'marker': 's'
    },
    'sans_alpha0': {
        'sans_enabled': True,
        'sans_temp': 0.0,  # Force uniform weighting
        'sans_num_negs': 16,
        'sans_temp_schedule': False,  # No schedule with α=0
        'description': 'SANS Parity (α=0, K=16, uniform)',
        'color': 'green',
        'marker': '^'
    }
}

print("Test Configurations:")
print("="*60)
for name, config in TEST_CONFIGS.items():
    print(f"  {name}: {config['description']}")
print("="*60)

## 3. Setup Dataset

In [ ]:
# Create dataset
if BASE_CONFIG['dataset'] == 'inverse':
    dataset = Inverse("train", BASE_CONFIG['rank'], BASE_CONFIG['ood'])
elif BASE_CONFIG['dataset'] == 'addition':
    dataset = Addition("train", BASE_CONFIG['rank'], BASE_CONFIG['ood'])
elif BASE_CONFIG['dataset'] == 'lowrank':
    dataset = LowRankDataset("train", BASE_CONFIG['rank'], BASE_CONFIG['ood'])
else:
    raise ValueError(f"Unknown dataset: {BASE_CONFIG['dataset']}")

validation_dataset = dataset
metric = 'mse'

print(f"Dataset: {BASE_CONFIG['dataset']}")
print(f"  Input dimension: {dataset.inp_dim}")
print(f"  Output dimension: {dataset.out_dim}")
print(f"  Dataset size: {len(dataset)}")

## 4. Unified Training Function

In [ ]:
def run_training(config_name, test_config, base_config, dataset, validation_dataset, capture_output=True):
    """
    Run training for a specific configuration
    
    Args:
        config_name: Name of the configuration (e.g., 'baseline', 'sans_normal')
        test_config: Specific test configuration parameters
        base_config: Base configuration parameters
        dataset: Training dataset
        validation_dataset: Validation dataset
        capture_output: Whether to capture console output for debugging
    
    Returns:
        dict: Training results and captured output
    """
    print("\n" + "="*60)
    print(f"RUNNING: {test_config['description']}")
    print("="*60)
    
    # Create results directory
    results_dir = Path(base_config['results_dir']) / config_name
    results_dir.mkdir(parents=True, exist_ok=True)
    
    # Merge configurations
    config = {**base_config, **test_config}
    
    # Save configuration
    with open(results_dir / 'config.json', 'w') as f:
        json.dump(config, f, indent=2)
    
    # Create model
    model = EBM(
        inp_dim=dataset.inp_dim,
        out_dim=dataset.out_dim,
    )
    model = DiffusionWrapper(model)
    
    # Capture output if requested
    captured_output = ""
    if capture_output:
        output_buffer = io.StringIO()
    
    try:
        if config.get('sans_enabled', False):
            # SANS configuration
            print(f"  SANS Config: α={config.get('sans_temp', 1.0)}, K={config.get('sans_num_negs', 16)}, "
                  f"schedule={config.get('sans_temp_schedule', True)}")
            
            # Create diffusion with SANS
            diffusion = GaussianDiffusion1D(
                model,
                seq_length=32,
                objective='pred_noise',
                timesteps=config['diffusion_steps'],
                sampling_timesteps=config['diffusion_steps'],
                supervise_energy_landscape=config['supervise_energy_landscape'],
                use_innerloop_opt=config['use_innerloop_opt'],
                show_inference_tqdm=False,
                # SANS parameters
                sans_enabled=True,
                sans_num_negs=config.get('sans_num_negs', 16),
                sans_temp=config.get('sans_temp', 2.0),
                sans_temp_schedule=config.get('sans_temp_schedule', True),
                sans_chunk=config.get('sans_chunk', 0),
                continuous=True  # For inverse dataset
            )
            
            # Create SANS trainer
            if capture_output:
                with redirect_stdout(output_buffer):
                    trainer = Trainer1D(
                        diffusion,
                        dataset,
                        train_batch_size=config['batch_size'],
                        validation_batch_size=min(256, config['batch_size']),
                        train_lr=config['learning_rate'],
                        train_num_steps=config['num_steps'],
                        gradient_accumulate_every=config['gradient_accumulate_every'],
                        ema_decay=config['ema_decay'],
                        data_workers=config['data_workers'],
                        amp=config['amp'],
                        metric='mse',
                        results_folder=str(results_dir),
                        cond_mask=False,
                        validation_dataset=validation_dataset,
                        save_and_sample_every=config['save_every'],
                        evaluate_first=False
                    )
                    trainer.train()
            else:
                trainer = Trainer1D(
                    diffusion,
                    dataset,
                    train_batch_size=config['batch_size'],
                    validation_batch_size=min(256, config['batch_size']),
                    train_lr=config['learning_rate'],
                    train_num_steps=config['num_steps'],
                    gradient_accumulate_every=config['gradient_accumulate_every'],
                    ema_decay=config['ema_decay'],
                    data_workers=config['data_workers'],
                    amp=config['amp'],
                    metric='mse',
                    results_folder=str(results_dir),
                    cond_mask=False,
                    validation_dataset=validation_dataset,
                    save_and_sample_every=config['save_every'],
                    evaluate_first=False
                )
                trainer.train()
        else:
            # Baseline iREM configuration
            print("  Using baseline iREM (no SANS)")
            
            if capture_output:
                with redirect_stdout(output_buffer):
                    trainer = iREMTrainer1D(
                        model,
                        dataset,
                        train_batch_size=config['batch_size'],
                        validation_batch_size=min(256, config['batch_size']),
                        train_lr=config['learning_rate'],
                        train_num_steps=config['num_steps'],
                        gradient_accumulate_every=config['gradient_accumulate_every'],
                        ema_decay=config['ema_decay'],
                        data_workers=config['data_workers'],
                        amp=config['amp'],
                        metric='mse',
                        results_folder=str(results_dir),
                        cond_mask=False,
                        validation_dataset=validation_dataset,
                        save_and_sample_every=config['save_every'],
                        evaluate_first=False
                    )
                    trainer.train()
            else:
                trainer = iREMTrainer1D(
                    model,
                    dataset,
                    train_batch_size=config['batch_size'],
                    validation_batch_size=min(256, config['batch_size']),
                    train_lr=config['learning_rate'],
                    train_num_steps=config['num_steps'],
                    gradient_accumulate_every=config['gradient_accumulate_every'],
                    ema_decay=config['ema_decay'],
                    data_workers=config['data_workers'],
                    amp=config['amp'],
                    metric='mse',
                    results_folder=str(results_dir),
                    cond_mask=False,
                    validation_dataset=validation_dataset,
                    save_and_sample_every=config['save_every'],
                    evaluate_first=False
                )
                trainer.train()
        
        print(f"\n✓ {config_name} training complete")
        
        if capture_output:
            captured_output = output_buffer.getvalue()
            
    except Exception as e:
        print(f"\n⚠ Training error for {config_name}: {str(e)}")
        if capture_output:
            captured_output = output_buffer.getvalue()
    
    return {
        'config_name': config_name,
        'results_dir': results_dir,
        'captured_output': captured_output
    }

print("✓ Training function defined")

## 5. Run All Test Configurations

In [ ]:
# Run all test configurations
results = {}

for config_name, test_config in TEST_CONFIGS.items():
    result = run_training(
        config_name=config_name,
        test_config=test_config,
        base_config=BASE_CONFIG,
        dataset=dataset,
        validation_dataset=validation_dataset,
        capture_output=(config_name.startswith('sans'))  # Capture output for SANS runs
    )
    results[config_name] = result

print("\n" + "="*60)
print("ALL TRAINING RUNS COMPLETE")
print("="*60)

## 6. Extract SANS Debug Information

In [ ]:
# Extract and display SANS debug information
for config_name, result in results.items():
    if 'sans' in config_name and result['captured_output']:
        print(f"\n{'='*60}")
        print(f"DEBUG OUTPUT: {TEST_CONFIGS[config_name]['description']}")
        print('='*60)
        
        # Extract SANS debug lines
        debug_lines = []
        for line in result['captured_output'].split('\n'):
            if '[SANS Debug]' in line or any(x in line for x in ['Neg energy:', 'Real energy:', 'Weight', 'Corr', 'Alpha', 'Loss components:']):
                debug_lines.append(line)
        
        if debug_lines:
            print("\nSample Debug Output (first occurrence):")
            # Find first complete debug block
            for i, line in enumerate(debug_lines):
                if '[SANS Debug]' in line and i + 10 < len(debug_lines):
                    for j in range(min(11, len(debug_lines) - i)):
                        print(debug_lines[i + j])
                    break
        else:
            print("No debug output captured (may need to increase logging frequency)")

## 7. Load and Analyze Metrics

In [ ]:
# Load metrics from all runs
metrics_data = {}

for config_name, result in results.items():
    metrics_path = result['results_dir'] / 'metrics.csv'
    
    if metrics_path.exists():
        df = pd.read_csv(metrics_path)
        df['config'] = config_name
        df['description'] = TEST_CONFIGS[config_name]['description']
        metrics_data[config_name] = df
        print(f"✓ Loaded metrics for {config_name}: {len(df)} steps")
    else:
        print(f"⚠ No metrics file for {config_name}")

# Combine all metrics
if metrics_data:
    all_metrics = pd.concat(list(metrics_data.values()), ignore_index=True)
    print(f"\n✓ Total metrics loaded: {len(all_metrics)} rows")
else:
    print("\n⚠ No metrics data available")

## 8. Parity Comparison Table

In [ ]:
# Create parity comparison table
def get_metrics_at_step(df, step, window=50):
    """Get metrics near a specific step with averaging window"""
    mask = (df['step'] >= step - window) & (df['step'] <= step + window)
    if mask.sum() > 0:
        return {
            'loss': df.loc[mask, 'loss'].mean(),
            'val_loss': df.loc[mask, 'val_loss'].dropna().mean() if 'val_loss' in df else None,
            'actual_step': df.loc[mask, 'step'].mean()
        }
    return {'loss': None, 'val_loss': None, 'actual_step': None}

# Build comparison table
comparison_steps = [1000, 5000]
comparison_data = []

for step in comparison_steps:
    row = {'Step': step}
    
    for config_name in ['baseline', 'sans_normal', 'sans_alpha0']:
        if config_name in metrics_data:
            metrics = get_metrics_at_step(metrics_data[config_name], step)
            
            if metrics['loss'] is not None:
                row[f"{config_name}_loss"] = metrics['loss']
                
                # Calculate difference from baseline
                if config_name != 'baseline' and 'baseline' in metrics_data:
                    baseline_metrics = get_metrics_at_step(metrics_data['baseline'], step)
                    if baseline_metrics['loss'] is not None:
                        diff = (metrics['loss'] - baseline_metrics['loss']) / baseline_metrics['loss'] * 100
                        row[f"{config_name}_diff%"] = diff
    
    comparison_data.append(row)

# Create and display comparison table
comparison_df = pd.DataFrame(comparison_data)

print("\n" + "="*80)
print("PARITY TEST RESULTS")
print("="*80)
print("\nLoss Comparison at Key Steps:")
print("-"*80)

# Format the table for display
display_data = []
for _, row in comparison_df.iterrows():
    display_row = [f"Step {int(row['Step'])}"]
    
    # Baseline
    if 'baseline_loss' in row:
        display_row.append(f"{row['baseline_loss']:.6f}")
    else:
        display_row.append("-")
    
    # SANS Normal
    if 'sans_normal_loss' in row:
        display_row.append(f"{row['sans_normal_loss']:.6f}")
        if 'sans_normal_diff%' in row:
            display_row.append(f"{row['sans_normal_diff%']:+.1f}%")
        else:
            display_row.append("-")
    else:
        display_row.extend(["-", "-"])
    
    # SANS Alpha=0
    if 'sans_alpha0_loss' in row:
        display_row.append(f"{row['sans_alpha0_loss']:.6f}")
        if 'sans_alpha0_diff%' in row:
            display_row.append(f"{row['sans_alpha0_diff%']:+.1f}%")
        else:
            display_row.append("-")
    else:
        display_row.extend(["-", "-"])
    
    display_data.append(display_row)

headers = ['Step', 'Baseline', 'SANS(α=2.0)', 'Diff%', 'SANS(α=0)', 'Diff%']
print(tabulate(display_data, headers=headers, tablefmt='grid'))

# Parity test conclusion
print("\n" + "="*80)
print("PARITY TEST ANALYSIS")
print("="*80)

if 'sans_alpha0_diff%' in comparison_df.columns:
    alpha0_diffs = comparison_df['sans_alpha0_diff%'].dropna()
    if len(alpha0_diffs) > 0:
        max_diff = alpha0_diffs.abs().max()
        avg_diff = alpha0_diffs.mean()
        
        print(f"\nSANS(α=0) vs Baseline:")
        print(f"  Average difference: {avg_diff:+.2f}%")
        print(f"  Maximum difference: {max_diff:.2f}%")
        
        if max_diff <= 2.0:
            print("\n✅ PARITY TEST PASSED")
            print("  SANS with α=0 matches baseline within 2%")
            print("  → The SANS codepath is equivalent to baseline")
            print("  → Issues are likely in the weighting/temperature schedule")
        else:
            print("\n⚠ PARITY TEST FAILED")
            print(f"  SANS with α=0 differs from baseline by {max_diff:.1f}%")
            print("  → The SANS codepath has structural differences")
            print("  → Check: negative term aggregation, loss mixing, or shape issues")

# Save comparison table
comparison_df.to_csv(Path(BASE_CONFIG['results_dir']) / 'parity_comparison.csv', index=False)
print(f"\n✓ Parity comparison saved to {BASE_CONFIG['results_dir']}/parity_comparison.csv")

## 9. Visualize Training Curves

In [ ]:
# Create comprehensive visualization
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('SANS Parity Test: Training Comparison', fontsize=16, y=1.02)

# 1. Training Loss Curves
ax = axes[0, 0]
for config_name in ['baseline', 'sans_normal', 'sans_alpha0']:
    if config_name in metrics_data:
        df = metrics_data[config_name]
        config = TEST_CONFIGS[config_name]
        ax.plot(df['step'], df['loss'], 
                label=config['description'],
                color=config['color'],
                alpha=0.8, linewidth=2)

ax.set_xlabel('Training Step')
ax.set_ylabel('Loss')
ax.set_title('Training Loss Comparison')
ax.legend(loc='best', fontsize=9)
ax.grid(True, alpha=0.3)
ax.set_yscale('log')

# Add vertical lines at comparison steps
for step in comparison_steps:
    ax.axvline(x=step, color='gray', linestyle='--', alpha=0.5)

# 2. Loss Difference from Baseline
ax = axes[0, 1]
if 'baseline' in metrics_data:
    baseline_df = metrics_data['baseline']
    
    for config_name in ['sans_normal', 'sans_alpha0']:
        if config_name in metrics_data:
            df = metrics_data[config_name]
            config = TEST_CONFIGS[config_name]
            
            # Interpolate to match baseline steps
            interp_loss = np.interp(baseline_df['step'], df['step'], df['loss'])
            diff_percent = (interp_loss - baseline_df['loss']) / baseline_df['loss'] * 100
            
            ax.plot(baseline_df['step'], diff_percent,
                    label=config['description'],
                    color=config['color'],
                    alpha=0.8, linewidth=2)
    
    ax.axhline(y=0, color='black', linestyle='-', alpha=0.5)
    ax.axhline(y=2, color='red', linestyle='--', alpha=0.3, label='±2% threshold')
    ax.axhline(y=-2, color='red', linestyle='--', alpha=0.3)
    
    ax.set_xlabel('Training Step')
    ax.set_ylabel('Difference from Baseline (%)')
    ax.set_title('Relative Performance vs Baseline')
    ax.legend(loc='best', fontsize=9)
    ax.grid(True, alpha=0.3)

# 3. Loss Components (if available)
ax = axes[1, 0]
for config_name in ['sans_normal', 'sans_alpha0']:
    if config_name in metrics_data:
        df = metrics_data[config_name]
        if 'loss_energy' in df.columns:
            config = TEST_CONFIGS[config_name]
            ax.plot(df['step'], df['loss_energy'],
                    label=f"{config_name} energy",
                    color=config['color'],
                    alpha=0.8, linewidth=1.5)

ax.set_xlabel('Training Step')
ax.set_ylabel('Energy Loss')
ax.set_title('Energy Loss Component')
ax.legend(loc='best', fontsize=9)
ax.grid(True, alpha=0.3)

# 4. Convergence Analysis
ax = axes[1, 1]
convergence_data = []

for config_name in ['baseline', 'sans_normal', 'sans_alpha0']:
    if config_name in metrics_data:
        df = metrics_data[config_name]
        config = TEST_CONFIGS[config_name]
        
        # Calculate smoothed derivative (rate of improvement)
        window = 100
        smoothed = df['loss'].rolling(window=window, min_periods=10).mean()
        rate = -smoothed.diff() / window  # Negative for improvement
        
        ax.plot(df['step'][window:], rate[window:],
                label=config['description'],
                color=config['color'],
                alpha=0.8, linewidth=1.5)

ax.set_xlabel('Training Step')
ax.set_ylabel('Improvement Rate')
ax.set_title('Training Convergence Rate')
ax.legend(loc='best', fontsize=9)
ax.grid(True, alpha=0.3)
ax.axhline(y=0, color='black', linestyle='-', alpha=0.5)

plt.tight_layout()
plt.savefig(Path(BASE_CONFIG['results_dir']) / 'parity_test_plots.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\n✓ Plots saved to {BASE_CONFIG['results_dir']}/parity_test_plots.png")

## 10. Final Summary and Recommendations

In [ ]:
print("\n" + "="*80)
print("PARITY TEST SUMMARY")
print("="*80)

print(f"\n📊 Dataset: {BASE_CONFIG['dataset']} (rank={BASE_CONFIG['rank']})")
print(f"⚙️  Training Steps: {BASE_CONFIG['num_steps']}")
print(f"🎯 Batch Size: {BASE_CONFIG['batch_size']}")

print("\n" + "-"*40)
print("KEY FINDINGS")
print("-"*40)

# Analyze parity test results
if 'sans_alpha0_diff%' in comparison_df.columns:
    alpha0_diffs = comparison_df['sans_alpha0_diff%'].dropna()
    if len(alpha0_diffs) > 0:
        max_diff = alpha0_diffs.abs().max()
        
        if max_diff <= 2.0:
            print("\n✅ SANS Implementation is Structurally Correct")
            print("   • SANS(α=0) matches baseline within acceptable margin")
            print("   • The negative sampling and aggregation logic is correct")
            print("   • Performance issues are in the weighting mechanism or temperature schedule")
            
            print("\n📝 Recommended Actions:")
            print("   1. Review temperature schedule (currently reversed)")
            print("   2. Tune base temperature (α) value")
            print("   3. Verify gradient flow through weighted negatives")
            print("   4. Check if loss_opt should be included")
        else:
            print("\n⚠ SANS Implementation Has Structural Issues")
            print(f"   • SANS(α=0) differs from baseline by {max_diff:.1f}%")
            print("   • The issue is NOT just in weighting")
            
            print("\n🔍 Areas to Investigate:")
            print("   1. Negative term aggregation (logsumexp vs mean)")
            print("   2. Loss component scaling (check loss_scale)")
            print("   3. Shape/dimension mismatches in energy computation")
            print("   4. Gradient accumulation differences")
            print("   5. Detach operations affecting gradient flow")

# Performance comparison
if 'baseline' in metrics_data and 'sans_normal' in metrics_data:
    baseline_final = metrics_data['baseline']['loss'].iloc[-1]
    sans_final = metrics_data['sans_normal']['loss'].iloc[-1]
    improvement = (baseline_final - sans_final) / baseline_final * 100
    
    print("\n" + "-"*40)
    print("PERFORMANCE COMPARISON")
    print("-"*40)
    print(f"\nFinal Loss @ {BASE_CONFIG['num_steps']} steps:")
    print(f"  Baseline:         {baseline_final:.6f}")
    print(f"  SANS (α=2.0):     {sans_final:.6f}")
    if 'sans_alpha0' in metrics_data:
        sans_alpha0_final = metrics_data['sans_alpha0']['loss'].iloc[-1]
        print(f"  SANS (α=0):       {sans_alpha0_final:.6f}")
    print(f"\n  SANS Improvement: {improvement:+.1f}%")

print("\n" + "="*80)
print("PARITY TEST COMPLETE")
print("="*80)

# List all output files
print("\n📁 Output Files:")
for file in Path(BASE_CONFIG['results_dir']).rglob('*'):
    if file.is_file() and not file.name.startswith('.'):
        print(f"   • {file.relative_to(BASE_CONFIG['results_dir'])}")